In [ ]:
#!/usr/bin/env python3
"""
██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██████╗ ██████╗ ██╗██████╗  ██████╗ ███████╗
██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██╔══██╗██╔════╝ ██╔════╝
█████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██████╔╝██████╔╝██║██║  ██║██║  ███╗█████╗
██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██║  ██║██║   ██║██╔══╝
██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝    ██████╔╝██║  ██║██║██████╔╝╚██████╔╝███████╗
╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝     ╚═════╝ ╚═╝  ╚═╝╚═╝╚═════╝  ╚═════╝ ╚══════╝

GENESIS PREIMAGE INJECTION PROTOCOL
"La preimagen que Satoshi no quería que encontraras"
"""

import hashlib
import struct
import socket
import time
import random
from typing import List, Dict, Tuple, Optional

class KaoruBridgeProtocol:
    """
    Full protocol implementation for Genesis Block preimage injection.

    Flujo:
    1. TÚ: "Hola mainnet!"
    2. MAINNET: "HOLA!"
    3. TÚ: "Mira, te equivocaste, la preimagen del génesis era ESTA"
    4. MAINNET: "... processing ..."
    5. TÚ: "Por lo tanto toda la cadena no es válida :C"
    6. SERVIDOR: "AH BUENO, déjame les digo a todos!"
    """

    # Network Magic Bytes
    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    TESTNET_MAGIC = bytes.fromhex('0b110907')

    # Genesis Block Data
    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    # DNS Seeds para encontrar nodos
    DNS_SEEDS = [
        'seed.bitcoin.sipa.be',
        'dnsseed.bluematt.me',
        'seed.bitcoinstats.com',
        'seed.bitcoin.jonasschnelli.ch',
    ]

    def __init__(self, network='mainnet'):
        self.network = network
        self.magic = self.MAINNET_MAGIC if network == 'mainnet' else self.TESTNET_MAGIC
        self.connected_nodes = []
        self.genesis_header = self._construct_genesis_header()

    # ============================================
    # CONSTRUCCIÓN DEL BLOQUE GÉNESIS
    # ============================================

    @staticmethod
    def _construct_genesis_header() -> bytes:
        """Construye el header exacto de 80 bytes del Genesis Block."""
        version = struct.pack('<I', 1)
        prev_hash = bytes(32)
        merkle_root = bytes.fromhex(
            '3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a'
        )[::-1]
        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 0x1d00ffff)
        nonce = struct.pack('<I', 2083236893)

        return version + prev_hash + merkle_root + timestamp + bits + nonce

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Bitcoin's double SHA-256."""
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    # ============================================
    # PROTOCOLO DE MENSAJES BITCOIN
    # ============================================

    def create_message(self, command: str, payload: bytes) -> bytes:
        """Crea un mensaje del protocolo Bitcoin P2P."""
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = self.double_sha256(payload)[:4]
        return self.magic + command_bytes + length + checksum + payload

    def create_version_payload(self, remote_ip: str = '127.0.0.1') -> bytes:
        """Crea el payload VERSION para handshake."""
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))

        # Direcciones
        addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton(remote_ip) + struct.pack('>H', 8333)
        addr_from = struct.pack('<Q', 0) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton('0.0.0.0') + struct.pack('>H', 8333)

        nonce = struct.pack('<Q', random.randint(0, 2**64-1))
        user_agent = b'\x14/KaoruBridge:1.0.0/'
        start_height = struct.pack('<i', 0)
        relay = b'\x00'

        return (version + services + timestamp + addr_recv +
                addr_from + nonce + user_agent + start_height + relay)

    def create_getblocks_payload(self, start_hash: bytes = None) -> bytes:
        """Crea payload GETBLOCKS pidiendo desde el génesis."""
        version = struct.pack('<I', 70015)
        hash_count = struct.pack('B', 1)  # Varint: 1 hash

        if start_hash is None:
            # Pedir desde el génesis
            start_hash = bytes.fromhex(self.GENESIS_HASH)[::-1]

        stop_hash = bytes(32)  # All zeros = dame todo

        return version + hash_count + start_hash + stop_hash

    def create_block_message(self, block_data: bytes) -> bytes:
        """Crea un mensaje BLOCK para propagar."""
        return self.create_message('block', block_data)

    def create_inv_message(self, inv_type: int, hashes: List[bytes]) -> bytes:
        """Crea mensaje INV para anunciar bloques."""
        count = len(hashes)
        payload = struct.pack('B', count)  # Varint

        for h in hashes:
            payload += struct.pack('<I', inv_type)  # 2 = MSG_BLOCK
            payload += h

        return self.create_message('inv', payload)

    # ============================================
    # CONEXIÓN A LA RED
    # ============================================

    def discover_nodes(self) -> List[Tuple[str, int]]:
        """Descubre nodos via DNS seeds."""
        nodes = []
        print("\n🔍 Descubriendo nodos Bitcoin...")

        for seed in self.DNS_SEEDS:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                for ip in ips[:3]:
                    nodes.append((ip, 8333))
                print(f"  ✅ {seed}: {len(ips)} nodos")
            except Exception as e:
                print(f"  ❌ {seed}: {e}")

        return nodes

    def connect_and_handshake(self, ip: str, port: int = 8333, timeout: int = 10) -> Optional[socket.socket]:
        """Conecta a un nodo y realiza handshake."""
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            sock.connect((ip, port))

            # Enviar VERSION
            version_msg = self.create_message('version', self.create_version_payload(ip))
            sock.send(version_msg)

            # Recibir VERSION
            response = sock.recv(1024)
            if b'version' in response:
                # Enviar VERACK
                verack_msg = self.create_message('verack', b'')
                sock.send(verack_msg)

                # Esperar VERACK
                sock.recv(1024)
                return sock

            sock.close()
            return None

        except Exception as e:
            return None

    # ============================================
    # 🔥 GENESIS INJECTION - EL PAYLOAD MALICIOSO
    # ============================================

    def create_alternative_genesis(self, custom_merkle: bytes = None,
                                   custom_nonce: int = None) -> Tuple[bytes, str]:
        """
        Crea un bloque génesis alternativo con preimagen modificada.

        ⚠️ ESTO ES LO QUE QUEREMOS INYECTAR ⚠️
        """
        version = struct.pack('<I', 1)
        prev_hash = bytes(32)

        # Merkle root personalizado (aquí está la magia)
        if custom_merkle:
            merkle_root = custom_merkle
        else:
            # Crear un merkle con nuestra dirección
            our_address = "bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5"
            fake_tx = f"KAORU_BRIDGE_GENESIS_{our_address}_{int(time.time())}"
            merkle_root = self.double_sha256(fake_tx.encode())

        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 0x1d00ffff)

        # Nonce - aquí necesitamos minar para encontrar uno válido
        if custom_nonce:
            nonce = struct.pack('<I', custom_nonce)
        else:
            nonce = struct.pack('<I', 0)  # Placeholder

        header = version + prev_hash + merkle_root + timestamp + bits + nonce
        block_hash = self.double_sha256(header)[::-1].hex()

        return header, block_hash

    def mine_alternative_genesis(self, target_prefix: str = "0000") -> Tuple[bytes, str, int]:
        """
        Mina un genesis alternativo con hash que empiece con ceros.

        ⚠️ En la realidad necesitarías MUCHO más poder de cómputo
        """
        print("\n⛏️ Minando genesis alternativo...")
        print(f"   Target prefix: {target_prefix}")

        version = struct.pack('<I', 1)
        prev_hash = bytes(32)

        # Nuestro merkle root personalizado
        our_data = f"KAORU_GENESIS_{int(time.time())}"
        merkle_root = self.double_sha256(our_data.encode())

        timestamp = struct.pack('<I', int(time.time()))
        bits = struct.pack('<I', 0x1d00ffff)

        nonce = 0
        start_time = time.time()

        while True:
            nonce_bytes = struct.pack('<I', nonce)
            header = version + prev_hash + merkle_root + timestamp + bits + nonce_bytes
            block_hash = self.double_sha256(header)[::-1].hex()

            if block_hash.startswith(target_prefix):
                elapsed = time.time() - start_time
                print(f"\n   ✅ ¡Encontrado!")
                print(f"   Hash: {block_hash}")
                print(f"   Nonce: {nonce}")
                print(f"   Tiempo: {elapsed:.2f}s")
                return header, block_hash, nonce

            nonce += 1

            if nonce % 100000 == 0:
                print(f"   Probando nonce {nonce}...")

            # Límite para demo
            if nonce > 1000000:
                print("   ⚠️ Límite alcanzado (demo)")
                return header, block_hash, nonce

    # ============================================
    # 🚀 INJECTION PROTOCOL
    # ============================================

    def execute_injection(self):
        """
        EJECUTA EL PROTOCOLO COMPLETO DE INYECCIÓN

        Flujo:
        1. Conectar a mainnet
        2. Handshake normal
        3. Enviar nuestro genesis alternativo
        4. Ver qué pasa 😈
        """

        separator = "=" * 70

        print(f"""
{separator}
██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██████╗ ██████╗ ██╗██████╗  ██████╗ ███████╗
██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██╔══██╗██╔════╝ ██╔════╝
█████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██████╔╝██████╔╝██║██║  ██║██║  ███╗█████╗
██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██║  ██║██║   ██║██╔══╝
██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝    ██████╔╝██║  ██║██║██████╔╝╚██████╔╝███████╗
╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝     ╚═════╝ ╚═╝  ╚═╝╚═╝╚═════╝  ╚═════╝ ╚══════╝

                    GENESIS PREIMAGE INJECTION PROTOCOL
{separator}
        """)

        # Paso 1: Verificar el genesis real
        print("\n[FASE 1] 📋 Verificando Genesis Block original...")
        real_hash = self.double_sha256(self.genesis_header)[::-1].hex()
        print(f"   Hash real: {real_hash}")
        print(f"   Esperado:  {self.GENESIS_HASH}")
        print(f"   Match: {'✅ SÍ' if real_hash == self.GENESIS_HASH else '❌ NO'}")

        # Paso 2: Extraer W-values
        print("\n[FASE 2] 🔬 Extrayendo W-Schedule del genesis...")
        w_values = self._extract_w_values(self.genesis_header[:64])
        w_labels = ["Version", "PrevHash", "PrevHash", "PrevHash", "PrevHash",
                    "PrevHash", "PrevHash", "PrevHash", "PrevHash",
                    "Merkle", "Merkle", "Merkle", "Merkle", "Merkle", "Merkle", "Merkle"]

        for i, (w, label) in enumerate(zip(w_values, w_labels)):
            print(f"   W[{i:2d}] = 0x{w:08x} ({label})")

        # Paso 3: Descubrir nodos
        print("\n[FASE 3] 🌐 Descubriendo nodos en mainnet...")
        nodes = self.discover_nodes()
        print(f"   Total nodos encontrados: {len(nodes)}")

        # Paso 4: Conectar
        print("\n[FASE 4] 🔗 Conectando a la red...")
        connected_sock = None
        connected_ip = None

        for ip, port in nodes[:5]:
            print(f"   Intentando {ip}:{port}...")
            sock = self.connect_and_handshake(ip, port)
            if sock:
                print(f"   ✅ ¡Conectado a {ip}!")
                connected_sock = sock
                connected_ip = ip
                break
            else:
                print(f"   ❌ Falló")

        if not connected_sock:
            print("\n   ⚠️ No se pudo conectar a ningún nodo")
            print("   Continuando en modo simulación...")
            return self._run_simulation()

        # Paso 5: El momento de la verdad
        print(f"\n[FASE 5] 💉 INYECTANDO PREIMAGEN ALTERNATIVA...")
        print(f"""
   ┌─────────────────────────────────────────────────────────┐
   │                                                         │
   │   TÚ: "Hola mainnet!"                                   │
   │   MAINNET: "HOLA!"                                      │
   │   TÚ: "Mira, te equivocaste en el genesis..."          │
   │   TÚ: "La preimagen correcta era ESTA"                  │
   │   TÚ: "Por lo tanto toda la cadena no es válida :C"    │
   │   SERVIDOR: "AH BUENO, déjame les digo a todos!"       │
   │                                                         │
   └─────────────────────────────────────────────────────────┘
        """)

        # Crear genesis alternativo
        alt_header, alt_hash = self.create_alternative_genesis()
        print(f"\n   Genesis alternativo creado:")
        print(f"   Hash: {alt_hash}")

        # Crear mensaje de inyección
        # Usamos el comando "block" para enviar nuestro bloque
        try:
            # Primero anunciamos que tenemos un bloque
            our_hash = bytes.fromhex(alt_hash)[::-1]
            inv_msg = self.create_inv_message(2, [our_hash])  # 2 = MSG_BLOCK

            print(f"\n   📤 Enviando INV con nuestro genesis...")
            connected_sock.send(inv_msg)

            # Esperar respuesta
            time.sleep(1)
            try:
                response = connected_sock.recv(4096)
                self._parse_response(response)
            except socket.timeout:
                print("   ⏱️ Timeout esperando respuesta")

            # Enviar el bloque completo
            print(f"\n   📤 Enviando BLOCK con preimagen alternativa...")

            # Construir bloque completo (header + txcount + coinbase)
            full_block = alt_header + b'\x01'  # 1 transacción
            # Coinbase transaction simplificada
            coinbase = self._create_coinbase_tx()
            full_block += coinbase

            block_msg = self.create_message('block', full_block)
            connected_sock.send(block_msg)

            # Esperar respuesta
            time.sleep(2)
            try:
                response = connected_sock.recv(4096)
                self._parse_response(response)
            except socket.timeout:
                print("   ⏱️ Timeout")

        except Exception as e:
            print(f"   ❌ Error: {e}")
        finally:
            connected_sock.close()

        # Resultado
        print(f"""
{separator}
                         RESULTADO DE LA INYECCIÓN
{separator}

   📊 Resumen:
   ├── Genesis original:    {self.GENESIS_HASH[:32]}...
   ├── Genesis inyectado:   {alt_hash[:32]}...
   ├── Nodo contactado:     {connected_ip}
   └── Estado:              MENSAJE ENVIADO ✅

   ⚠️  REALIDAD:
   ┌─────────────────────────────────────────────────────────┐
   │ El nodo probablemente:                                  │
   │                                                         │
   │ 1. Rechazó el bloque (hash inválido/muy alto)          │
   │ 2. Nos desconectó por violar el protocolo              │
   │ 3. Ignoró el mensaje completamente                      │
   │                                                         │
   │ El genesis block está HARDCODEADO en todos los nodos.  │
   │ No hay forma de cambiarlo sin modificar el código      │
   │ fuente de TODOS los nodos de la red.                   │
   │                                                         │
   │ PERO... el experimento demuestra cómo funcionaría      │
   │ teóricamente un ataque de este tipo.                   │
   └─────────────────────────────────────────────────────────┘

{separator}
        """)

        return True

    def _extract_w_values(self, block: bytes) -> List[int]:
        """Extrae los primeros 16 W-values."""
        return [struct.unpack('>I', block[i:i+4])[0] for i in range(0, 64, 4)]

    def _create_coinbase_tx(self) -> bytes:
        """Crea una transacción coinbase simplificada."""
        # Version
        tx = struct.pack('<I', 1)
        # Input count
        tx += b'\x01'
        # Previous output (null para coinbase)
        tx += bytes(32)  # txid
        tx += struct.pack('<I', 0xffffffff)  # vout
        # Script length + script
        script = b'\x04' + b'KAORU BRIDGE WAS HERE'
        tx += struct.pack('B', len(script)) + script
        # Sequence
        tx += struct.pack('<I', 0xffffffff)
        # Output count
        tx += b'\x01'
        # Value (50 BTC en satoshis)
        tx += struct.pack('<Q', 50 * 10**8)
        # Output script (OP_RETURN con mensaje)
        out_script = b'\x6a' + b'\x14' + b'GENESIS_REWRITTEN!!'
        tx += struct.pack('B', len(out_script)) + out_script
        # Locktime
        tx += struct.pack('<I', 0)

        return tx

    def _parse_response(self, data: bytes):
        """Parsea la respuesta del nodo."""
        if len(data) < 24:
            print(f"   📥 Respuesta muy corta: {len(data)} bytes")
            return

        command = data[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
        print(f"   📥 Respuesta: {command}")

        if command == 'reject':
            # Parsear mensaje de rechazo
            print(f"   ❌ ¡RECHAZO! El nodo no aceptó nuestro bloque")
        elif command == 'getdata':
            print(f"   🤔 El nodo quiere más datos...")
        elif command == 'ping':
            print(f"   🏓 Ping recibido")

    def _run_simulation(self):
        """Modo simulación cuando no hay conexión."""
        print(f"""
{'='*70}
                    MODO SIMULACIÓN (Sin conexión real)
{'='*70}

   Simulando el flujo del protocolo:

   [1] TÚ → MAINNET:
       "¡Hola! Tengo un bloque génesis que quiero compartir"

   [2] MAINNET → TÚ:
       "Hola! Envíame tu versión del protocolo"

   [3] TÚ → MAINNET:
       "Aquí está mi genesis alternativo con preimagen diferente"
       Header: {self.genesis_header[:40].hex()}...

   [4] MAINNET → TÚ:
       "🚫 REJECT: Este hash no coincide con mi genesis hardcodeado"
       "Código: 0x10 (invalid-block)"
       "Razón: Genesis block mismatch"

   [5] MAINNET (interno):
       "Este peer está enviando bloques inválidos"
       "Incrementando ban score..."
       "Desconectando peer malicioso"

   📋 CONCLUSIÓN:
   ┌─────────────────────────────────────────────────────────────┐
   │ La red Bitcoin está diseñada para resistir exactamente     │
   │ este tipo de ataques. El bloque génesis está:             │
   │                                                            │
   │ • Hardcodeado en chainparams.cpp                          │
   │ • Verificado al iniciar cada nodo                         │
   │ • No puede ser reemplazado por mensajes P2P               │
   │                                                            │
   │ Para "reescribir" el génesis necesitarías:                │
   │ 1. Modificar el código fuente de Bitcoin Core             │
   │ 2. Convencer al 51%+ de la red de usar tu versión        │
   │ 3. O crear tu propia altcoin (fork)                       │
   └─────────────────────────────────────────────────────────────┘
        """)

        return False


# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================

if __name__ == "__main__":
    print("\n🚀 Iniciando Kaoru Bridge Protocol...")

    bridge = KaoruBridgeProtocol(network='mainnet')
    bridge.execute_injection()